In [ ]:
%pip install semantic-router==0.0.37
%pip install sentence-transformers
%pip install chromadb
%pip install pypdf
%pip install langchain-community

In [ ]:
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

encoder.encode("Hello, World!")

In [2]:
import chromadb

client = chromadb.PersistentClient(
    path="/Users/anavbo/Desktop/Personal/skin-cancer-detection-app/RAG/chromadb",
)

collection = client.create_collection("skin-cancer")

In [13]:
import chromadb.utils.embedding_functions as embedding_functions
chroma_encode = embedding_functions.CohereEmbeddingFunction(
    api_key=os.environ["COHERE_API_KEY"], model_name="embed-english-v3.0"
)

Document(metadata={'source': '/Users/anavbo/Desktop/Personal/skin-cancer-detection-app/RAG/documents/Actinic keratosis - Diagnosis & treatment - Mayo Clinic.pdf', 'page': 0}, page_content="Actinic keratosis\nDiagnosis\nYour health care provider will likely be able to determine whether you have an actinic\nkeratosis simply by looking at your skin. If there's any doubt, your health care\nprovider may do other tests, such as a skin biopsy. During a skin biopsy, a small\nsample of skin is taken for analysis in a lab. A biopsy can usually be done in a clinic\nafter a numbing injection.\nEven after treatment for actinic keratosis, your health care provider might suggest\nthat you have your skin checked at least once a year for signs of skin cancer.\nTreatment\nAn actinic keratosis sometimes disappears on its own but might return after more\nsun exposure. It's hard to tell which actinic keratoses will develop into skin cancer, so\nthey're usually removed as a precaution.\nMedicines\nIf you ha

In [26]:
document = ""
for doc in documents:
    document += doc.page_content + "\n"

In [63]:
# semantic chunking
from semantic_router.splitters import RollingWindowSplitter
from semantic_router.utils.logger import logger
import os
from semantic_router.encoders import CohereEncoder

os.environ["COHERE_API_KEY"] = "kUYLefL4SNo36iNh5tF2H1gGi5XfZyTbOBJNknt1"
logger.setLevel("WARNING")
encoder = CohereEncoder()
splitter = RollingWindowSplitter(
    encoder=encoder,
    min_split_tokens=50,
    max_split_tokens=400,
    window_size=2,
    enable_statistics=False,
)

In [64]:
from pypdf import PdfReader
import os
import math 
import uuid

pdf_paths = os.listdir(path)

chunks = []
metadatas = []
ids = []
for doc_path in pdf_paths:
    documents = PdfReader(f"{path}/{doc_path}")
    document = ""
    for doc in documents.pages:
        document += doc.extract_text() + "\n"

    splits = splitter([document])
    for i in range(len(splits)):
        chunk = " ".join(splits[i].docs)
        if i == 0:
            metadata = {
                "source": doc_path,
                "pre_chunk": None,
                "post_chunk": splits[i+1].docs[-1]
            }
        elif i == len(splits) - 1:
            metadata = {
                "source": doc_path,
                "pre_chunk": splits[i-1].docs[-1],
                "post_chunk": None
            }
        else:
            metadata = {
                "source": doc_path,
                "pre_chunk": splits[i-1].docs[-1],
                "post_chunk":  splits[i+1].docs[0]
            }
        chunks.append(chunk)
        metadatas.append(metadata)
        ids.append(str(uuid.uuid4().hex))

In [62]:
len(chunks)

750